------------------------------------------------

Necessary imports

In [9]:
import pandas as pd
import os 
import glob
! pip install pandasql
from pandasql import sqldf
import json
import warnings
warnings.filterwarnings('ignore')

In [10]:
current_directory = os.getcwd()
mydir = current_directory
file_list = glob.glob(mydir + "/*.csv")

In [11]:
file_list

['c:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\admissions.csv',
 'c:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\d_labitems.csv',
 'c:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\edstays.csv',
 'c:\\Users\\junin\\Documents\\Data-Science\\MIMICAnalysis\\labevents.csv']

In [12]:
list_df = []
for i in file_list:
    df = pd.read_csv(i)
    list_df.append(df)

In [13]:
df_admissions = list_df[0]
df_labitems = list_df[1]
df_edstays = list_df[2]
df_labevents = list_df[3]

## Reading the data

In [14]:
df_admissions.head(5)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24T14:38:00,2196-03-04T14:02:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2196-02-24T12:15:00,2196-02-24T17:07:00,0
1,10009628,25926192,2153-09-17T17:08:00,2153-09-25T13:20:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO,NaN,NaN,0
2,10018081,23983182,2134-08-18T02:02:00,2134-08-23T19:35:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17T16:24:00,2134-08-18T03:15:00,0
3,10006053,22942076,2111-11-13T23:39:00,2111-11-15T17:20:00,2111-11-15T17:20:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04T18:46:00,2113-08-06T20:57:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0


In [15]:
df_edstays.head(5)

,subject_id,hadm_id,stay_id,intime,outtime
0,10014729,23300884.0,32435236,2125-03-19T12:36:00,2125-03-19T16:59:47
1,10004235,24181354.0,36583389,2196-02-24T12:15:00,2196-02-24T17:07:00
2,10026255,22059910.0,30926886,2201-07-07T12:31:00,2201-07-07T19:40:00
3,10026255,20437651.0,36948716,2200-09-17T18:38:00,2200-09-18T00:57:00
4,10007058,22954658.0,35770266,2167-11-07T17:57:00,2167-11-07T20:22:00


In [16]:
df_labevents.head(5)

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,183881,10014354,29600294.0,1808066,51277,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.4,%,10.5,15.5,NaN,ROUTINE,NaN
1,183883,10014354,29600294.0,1808066,51301,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.3,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
2,183888,10014354,29600294.0,1808066,52167,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.7,fL,35.1,46.3,abnormal,ROUTINE,NaN
3,183870,10014354,29600294.0,1808066,51249,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.1,g/dL,32.0,37.0,abnormal,ROUTINE,NaN
4,183865,10014354,29600294.0,1808066,51222,2148-08-16 00:00:00,2148-08-16 01:30:00,9.2,9.2,g/dL,13.7,17.5,abnormal,ROUTINE,NaN


In [17]:
df_labitems.head(5)

,itemid,label,fluid,category,loinc_code
0,52016,Abz,Blood,Blood Gas,NaN
1,52017,"Albumin, Blood",Blood,Blood Gas,NaN
2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,NaN
3,52018,Assist/Control,Blood,Blood Gas,NaN
4,50802,Base Excess,Blood,Blood Gas,NaN


 #  <span style="color:blue">  1.	What is the percentage of patients admitted to the hospital that have never been to the ED? </span>

In [18]:
# We have 100 patients in admissions table 
total_patients = len(df_admissions['subject_id'].unique())
print(total_patients)

100


In [19]:
#First i got all ED admissions
df_emergency_room = df_admissions.query('admission_location=="EMERGENCY ROOM"')

In [20]:
#After that I removed all admissions that contain the type ED from the admissions table
df_remove = df_admissions[~df_admissions.subject_id.isin(df_emergency_room.subject_id)]

In [21]:
#In the sequence of the table that contains the different admissions of the ED type, the filter of single patients was performed
total_dif = len(df_remove['subject_id'].unique())

In [22]:
#Calculation: of patients admitted to the hospital that have never been to the ED
porcent = (total_dif/total_patients)*100
print(f'Percentage of patients admitted to the hospital that have never been to the ED is {porcent} %')

Percentage of patients admitted to the hospital that have never been to the ED is 39.0 %


 #  <span style="color:blue">   2.		For each patient, generate a json with the hospital admission data. </span>

* My primary interpretation was to store all patients in a single JSON file, in which they can be identified by their respective ids/subject_id.


* However, there is also the possibility of creating a JSON file for each patient. This can be easily performed with simple coding changes.

##### I created two JSON files. In simple_admission.json only the data contained in the admissions table is present.

In [23]:
list_att = df_admissions.columns[1:]
d = (df_admissions.groupby(['subject_id'])[list_att]
       .apply(lambda x: x.to_dict('r')).to_json())
simple_json_object = json.loads(d)

In [24]:
for key in simple_json_object.keys():
    json_patient = simple_json_object[key]
    with open(f'{mydir}\\simple_admission_patients\\{key}.json', 'w') as fp:
        json.dump(json_patient, fp)

In [25]:
with open('simple_admission.json', 'w') as fp:
    json.dump(simple_json_object, fp)

In [26]:
##### The full admission.json contains all the exam results, stay that inherit the admissions of the patients through the relationships of the tables: df_edstays, df_labevents, and df_labitems.

In [27]:
query = """SELECT   df_admissions.subject_id,	df_admissions.hadm_id,	df_admissions.admittime,	df_admissions.dischtime,	df_admissions.deathtime,	df_admissions.admission_type,	df_admissions.admission_location,	df_admissions.discharge_location,	df_admissions.insurance,	df_admissions.language,	df_admissions.marital_status,	df_admissions.ethnicity,	df_admissions.edregtime,	df_admissions.edouttime,	df_admissions.hospital_expire_flag,

df_edstays.stay_id, df_edstays.intime stay_intime, df_edstays.outtime stay_outime,

df_labevents.labevent_id, df_labevents.specimen_id labE_specimen_id, df_labevents.itemid labE_itemid,	df_labevents.charttime labE_charttime,	df_labevents.storetime labE_storetime,	df_labevents.value labE_value,	df_labevents.valuenum labE_valuenum,	df_labevents.valueuom labE_valueuom,	df_labevents.ref_range_lower labE_ref_range_lower,	df_labevents.ref_range_upper labE_ref_range_upper,	df_labevents.flag labE_flag,	df_labevents.priority labE_priority,	df_labevents.comments labE_comments,


df_labitems.label labI_label,	df_labitems.fluid labI_fluid,	df_labitems.category labI_category,	df_labitems.loinc_code labI_loinc_code


from  df_admissions 


LEFT JOIN df_edstays on df_edstays.hadm_id = df_admissions.hadm_id

LEFT JOIN df_labevents on df_labevents.hadm_id = df_admissions.hadm_id

INNER JOIN df_labitems on df_labitems.itemid = df_labevents.itemid;

"""

df_query_resul =sqldf(query, globals())

In [28]:
list_att = df_query_resul.columns[1:]
d = (df_query_resul.groupby(['subject_id'])[list_att]
       .apply(lambda x: x.to_dict('r')).to_json())
full_json_object = json.loads(d)

In [29]:
for key in full_json_object.keys():
    json_patient = full_json_object[key]
    with open(f'{mydir}\\full_admission_patients\\{key}.json', 'w') as fp:
        json.dump(json_patient, fp)

In [30]:
with open('full_admission.json', 'w') as fp:
    json.dump(full_json_object, fp)

  #  <span style="color:blue">  3.	Check the performance of albumin in predicting in-hospital mortality. </span>
* a.	Use only each patient’s first admission to the hospital; and
* b.	Only the Albumin results from the first 24h from hospital admission.

 ##### Before performing the queries, it is necessary to perform a summary of the initial data.
* Total unique patients who had admissions: 100
* Total mortality considering only the first admission and regardless of albumin:  5

##### In interpreting the resolution of question 3, I considered a cumulative and dependency solution between alternatives A and B.

####  a. Use only each patient’s first admission to the hospital

#### Using each dataframe as a table in SQL language over a Sqlite

In [31]:
first_query ="""

SELECT Q1.*
FROM

(SELECT  *, ROW_NUMBER ()  OVER (PARTITION BY subject_id ORDER BY STRFTIME('%Y-%m-%d %H:%M:%S', admittime) ASC)  RowNum

from 

df_admissions) Q1 

WHERE Q1. RowNum = 1

"""
df_first_admission =sqldf(first_query, globals())

In [32]:
df_first_admission

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,RowNum
0,10000032,22595853,2180-05-06T22:23:00,2180-05-07T17:15:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06T19:17:00,2180-05-06T23:30:00,0,1
1,10001217,24597018,2157-11-18T22:56:00,2157-11-25T18:00:00,None,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18T17:38:00,2157-11-19T01:24:00,0,1
2,10001725,25563031,2110-04-11T15:08:00,2110-04-14T15:00:00,None,EW EMER.,PACU,HOME,Other,ENGLISH,MARRIED,WHITE,None,None,0,1
3,10002428,25797028,2155-07-14T19:15:00,2155-07-15T18:37:00,None,EU OBSERVATION,EMERGENCY ROOM,None,Medicare,ENGLISH,WIDOWED,WHITE,2155-07-14T16:58:00,2155-07-14T20:04:00,0,1
4,10002495,24982426,2141-05-22T20:17:00,2141-05-29T17:41:00,None,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,UNKNOWN,None,None,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10038999,27189241,2131-05-22T21:49:00,2131-06-04T13:43:00,None,OBSERVATION ADMIT,EMERGENCY ROOM,HOME,Medicare,ENGLISH,SINGLE,WHITE,2131-05-22T20:33:00,2131-05-22T21:28:00,0,1
96,10039708,20572787,2138-10-30T23:30:00,2138-11-06T23:30:00,None,EW EMER.,EMERGENCY ROOM,AGAINST ADVICE,Other,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2138-10-30T17:27:00,2138-10-31T00:19:00,0,1
97,10039831,26924951,2115-12-28T07:15:00,2116-01-02T14:34:00,None,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,UNABLE TO OBTAIN,None,None,0,1
98,10039997,24294180,2134-09-07T12:00:00,2134-09-11T13:10:00,None,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,None,None,0,1


####  b. Only the Albumin results from the first 24h from hospital admission.

#### To answer the criteria of question b. The queries were divided to facilitate the understanding of the solution presented.
#### Each step results in a dataframe. 

* 1: Selecting admission with albumin results --> df_albumin_lab
* 2: Filtering lab events in the first 24h --> df_time_event_lab
* 3: Removing outliers --> df_clear_admission
* 4: Counting number of deaths

#### 1) Select only the admissions where lab_events is Albumin, joining the tables: first_admission, df_labevents and df_labitems

In [33]:
query_only_albumin_lab ="""

SELECT *
FROM
df_first_admission

JOIN df_labevents ON df_labevents.subject_id = df_first_admission.subject_id and df_labevents.hadm_id = df_first_admission.hadm_id


JOIN df_labitems ON df_labitems.itemid = df_labevents.itemid

where upper(df_labitems.label) like '%ALBUMIN%'


"""
df_albumin_lab =sqldf(query_only_albumin_lab, globals())

In [34]:
df_albumin_lab

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,...,ref_range_lower,ref_range_upper,flag,priority,comments,itemid,label,fluid,category,loinc_code
0,10002495,24982426,2141-05-22T20:17:00,2141-05-29T17:41:00,None,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,...,3.5,5.2,abnormal,ROUTINE,None,50862,Albumin,Blood,Chemistry,1751-7
1,10002495,24982426,2141-05-22T20:17:00,2141-05-29T17:41:00,None,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,...,3.5,5.2,abnormal,STAT,None,50862,Albumin,Blood,Chemistry,1751-7
2,10003400,26090619,2134-06-06T02:25:00,2134-06-07T15:05:00,None,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,...,3.5,5.2,abnormal,ROUTINE,None,50862,Albumin,Blood,Chemistry,1751-7
3,10004235,24181354,2196-02-24T14:38:00,2196-03-04T14:02:00,None,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,...,3.5,5.2,abnormal,STAT,None,50862,Albumin,Blood,Chemistry,1751-7
4,10004235,24181354,2196-02-24T14:38:00,2196-03-04T14:02:00,None,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,...,3.5,5.2,abnormal,ROUTINE,None,50862,Albumin,Blood,Chemistry,1751-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,10007818,22987108,2146-06-10T16:37:00,2146-07-12T00:00:00,2146-07-12T20:50:00,DIRECT EMER.,PHYSICIAN REFERRAL,DIED,Medicare,ENGLISH,...,NaN,NaN,None,STAT,None,50835,"Albumin, Ascites",Ascites,Chemistry,None
124,10007818,22987108,2146-06-10T16:37:00,2146-07-12T00:00:00,2146-07-12T20:50:00,DIRECT EMER.,PHYSICIAN REFERRAL,DIED,Medicare,ENGLISH,...,NaN,NaN,None,ROUTINE,None,50835,"Albumin, Ascites",Ascites,Chemistry,None
125,10007818,22987108,2146-06-10T16:37:00,2146-07-12T00:00:00,2146-07-12T20:50:00,DIRECT EMER.,PHYSICIAN REFERRAL,DIED,Medicare,ENGLISH,...,NaN,NaN,None,STAT,None,50835,"Albumin, Ascites",Ascites,Chemistry,None
126,10038081,20755971,2115-09-27T20:40:00,2115-10-12T00:00:00,2115-10-12T22:20:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Other,?,...,NaN,NaN,None,ROUTINE,LESS THAN 1.0.,50835,"Albumin, Ascites",Ascites,Chemistry,None


####  2) Filtering admissions respecting the criteria of lab events that were held after 24 hours of admission

##### The attributes used to count the hours were "admittime" from the admission table and "charttime" from the labevents table

* Q1: Internal query to get hours via julianday function ---> generating "time_event_lab" attribute: hours count
* Q2: External query to return admissions that are within the requested range



In [35]:
query_time_event_lab="""
SELECT Q1.*
FROM
(SELECT df_albumin_lab.*
   , CAST((julianday(charttime) - julianday(admittime))*24 AS real) AS time_event_lab
   
FROM df_albumin_lab) Q1 

WHERE Q1.time_event_lab <=24;

"""
df_time_event_lab =sqldf(query_time_event_lab, globals())

In [36]:
df_time_event_lab

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,...,ref_range_lower,ref_range_upper,flag,priority,comments,label,fluid,category,loinc_code,time_event_lab
0,10004720,22081550.0,2186-11-12T18:01:00,2186-11-17T18:30:00,2186-11-17T18:30:00,EW EMER.,INFORMATION NOT AVAILABLE,DIED,Medicare,ENGLISH,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,13.516667
1,10004720,22081550.0,2186-11-12T18:01:00,2186-11-17T18:30:00,2186-11-17T18:30:00,EW EMER.,INFORMATION NOT AVAILABLE,DIED,Medicare,ENGLISH,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,20.466667
2,10005817,20626031.0,2132-12-12T01:43:00,2132-12-20T15:04:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,ENGLISH,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,1.416667
3,10005866,26158160.0,2146-06-06T00:50:00,2146-06-09T16:45:00,None,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,-2.166667
4,10007818,22987108.0,2146-06-10T16:37:00,2146-07-12T00:00:00,2146-07-12T20:50:00,DIRECT EMER.,PHYSICIAN REFERRAL,DIED,Medicare,ENGLISH,...,3.5,5.2,None,STAT,None,Albumin,Blood,Chemistry,1751-7,2.983333
5,10007928,20338077.0,2129-04-05T22:56:00,2129-04-11T17:25:00,None,EW EMER.,EMERGENCY ROOM,HOME,Other,ENGLISH,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,18.316667
6,10009628,25926192.0,2153-09-17T17:08:00,2153-09-25T13:20:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,...,3.5,5.2,None,STAT,None,Albumin,Blood,Chemistry,1751-7,2.366667
7,10012552,27089790.0,2140-03-22T17:18:00,2140-03-30T14:10:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,ENGLISH,...,3.5,5.2,None,STAT,None,Albumin,Blood,Chemistry,1751-7,3.116667
8,10016150,29374560.0,2142-05-10T15:05:00,2142-05-15T19:49:00,None,EW EMER.,PROCEDURE SITE,REHAB,Medicare,ENGLISH,...,3.5,5.2,None,ROUTINE,None,Albumin,Blood,Chemistry,1751-7,16.166667
9,10016810,20973395.0,2185-06-16T01:31:00,2185-06-21T15:55:00,None,EW EMER.,EMERGENCY ROOM,HOME,Other,ENGLISH,...,3.5,5.2,abnormal,ROUTINE,None,Albumin,Blood,Chemistry,1751-7,23.600000


####  3) Checking inconsistency

##### After filtering the hours of the lab events, it was verified that it contained data in which the lab_event was performed before the date of admission of hospitalization. It can be classified as an inconsistency in the data, or an alert to report to professionals. 

##### However, for the resolution sequence, the admission: 26158160.0 of the patient: 10005866 was discarded from the count


In [37]:
df_clear_admission = df_time_event_lab.copy()

In [38]:
df_clear_admission.drop(df_clear_admission.loc[df_clear_admission['time_event_lab']<0].index, inplace=True)

### Final table of queries requested, but still needs to perform operations to obtain final conclusions on single-patient mortality

In [39]:
df_clear_admission.reset_index()

,index,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,ref_range_lower,ref_range_upper,flag,priority,comments,label,fluid,category,loinc_code,time_event_lab
0,0,10004720,22081550.0,2186-11-12T18:01:00,2186-11-17T18:30:00,2186-11-17T18:30:00,EW EMER.,INFORMATION NOT AVAILABLE,DIED,Medicare,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,13.516667
1,1,10004720,22081550.0,2186-11-12T18:01:00,2186-11-17T18:30:00,2186-11-17T18:30:00,EW EMER.,INFORMATION NOT AVAILABLE,DIED,Medicare,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,20.466667
2,2,10005817,20626031.0,2132-12-12T01:43:00,2132-12-20T15:04:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,1.416667
3,4,10007818,22987108.0,2146-06-10T16:37:00,2146-07-12T00:00:00,2146-07-12T20:50:00,DIRECT EMER.,PHYSICIAN REFERRAL,DIED,Medicare,...,3.5,5.2,None,STAT,None,Albumin,Blood,Chemistry,1751-7,2.983333
4,5,10007928,20338077.0,2129-04-05T22:56:00,2129-04-11T17:25:00,None,EW EMER.,EMERGENCY ROOM,HOME,Other,...,3.5,5.2,abnormal,STAT,None,Albumin,Blood,Chemistry,1751-7,18.316667
5,6,10009628,25926192.0,2153-09-17T17:08:00,2153-09-25T13:20:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,...,3.5,5.2,None,STAT,None,Albumin,Blood,Chemistry,1751-7,2.366667
6,7,10012552,27089790.0,2140-03-22T17:18:00,2140-03-30T14:10:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,...,3.5,5.2,None,STAT,None,Albumin,Blood,Chemistry,1751-7,3.116667
7,8,10016150,29374560.0,2142-05-10T15:05:00,2142-05-15T19:49:00,None,EW EMER.,PROCEDURE SITE,REHAB,Medicare,...,3.5,5.2,None,ROUTINE,None,Albumin,Blood,Chemistry,1751-7,16.166667
8,9,10016810,20973395.0,2185-06-16T01:31:00,2185-06-21T15:55:00,None,EW EMER.,EMERGENCY ROOM,HOME,Other,...,3.5,5.2,abnormal,ROUTINE,None,Albumin,Blood,Chemistry,1751-7,23.600000
9,10,10018423,29366372.0,2167-05-03T21:24:00,2167-05-11T12:57:00,None,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,3.5,5.2,None,ROUTINE,None,Albumin,Blood,Chemistry,1751-7,19.933333


####  4) Counting number of deaths

##### This indicates whether the patient died within the given hospitalization. 
#####  1: indicates death in the hospital
#####  0: indicates survival to hospital discharge.

### Total unique patients who had admissions: 32 - Considering the final results - provides from requirements (a and b)

In [40]:
print(f" The number of records is {len(df_clear_admission)}, but the number of patients is {len(df_clear_admission['subject_id'].unique())}")

 The number of records is 36, but the number of patients is 32


#### Number of unique patients who did not die - FINAL Result

In [41]:
#df_clear_admission.notna()
admission_no_death_final =df_clear_admission[df_clear_admission['deathtime'].isnull()]
print(f"Unique number of patients who did not die: {len(admission_no_death_final['subject_id'].unique())}")

Unique number of patients who did not die: 27


#### Number of unique patients who died - FINAL Result

In [42]:
admission_death_final =df_clear_admission[df_clear_admission['deathtime'].notnull()]
print(f"Unique number of patients who died: {len(admission_death_final['subject_id'].unique())}")

Unique number of patients who died: 5


# Formalizing the performance of albumin in predicting in-hospital mortality

### Total unique patients who had admissions: 100 - Considering the admission table with the first admission of each patient

In [43]:
print(f"Total unique patients who had admissions: {len(df_first_admission['subject_id'].unique())}")

Total unique patients who had admissions: 100


#### Number of unique patients who did not die - FIRST Admission

In [44]:
#df_clear_admission.notna()
admission_no_death_no_filter =df_first_admission[df_first_admission['deathtime'].isnull()]
print(f"Unique number of patients who did not die: {len(admission_no_death_no_filter['subject_id'].unique())}")

Unique number of patients who did not die: 95


#### Number of unique patients who died - FIRST Admission

In [45]:
admission_death_no_filter =df_first_admission[df_first_admission['deathtime'].notnull()]
print(f"Unique number of patients who died: {len(admission_death_no_filter['subject_id'].unique())}")

Unique number of patients who died: 5


### Summarizing

* Total unique patients who had admissions: 100
    * NO-FILTER albumin --- Unique number of patients who did not die: 95
    * NO-FILTER albumin --- Unique number of patients who  died: 5



* Total unique patients who had admissions apply all condiction(a and b): 32
    * Unique number of patients who did not die: 27
    * Unique number of patients who  died: 5


### Survival and mortality rate not considering only albumin tests

In [46]:
survival_rate =    ( len(admission_no_death_no_filter) / len(df_first_admission)) *100
print(f"The survival rate not considering albumin tests is: {survival_rate} %")

The survival rate not considering albumin tests is: 95.0 %


In [47]:
mortality_rate = (len(admission_death_no_filter) /  len(df_first_admission)) *100
print(f"The mortality rate not considering albumin tests is: {mortality_rate} %")

The mortality rate not considering albumin tests is: 5.0 %


### Survival and mortality rate considering albumin tests

In [48]:
survival_rate_final = ( len(admission_no_death_final) / len(df_clear_admission)) *100
print(f"{survival_rate_final} %")

75.0 %


In [49]:
mortality_rate_final = (len(admission_death_final) /  len(df_clear_admission)) *100
print(f"{mortality_rate_final} %")

25.0 %


### Final Insights

After analyzing the data of admissions, exams, stays.
The final table --> "df_clear_admission" with the exam results (lab_events) of admissions contains 36 records with 32 unique patients.

We can notice that without performing the requested filters in relation to the exclusivity of the presence of the albumin test. The probability of survival is 95%, and 5% for mortality. When we apply the filter of the existence of albumin tests, the percentage of survival drops to 75%, and mortality increases to 25%. It is possible to observe that the presence of the albumin test has a substantial impact on mortality, increasing the probability of death by 20%. By analyzing the data, it was also possible to notice that all patients who had deaths for both filters (not only the albumin test and only the albumin test) had performed tests related to the albumin test.


* By analyzing the data, it was also possible to notice that all patients who had deaths for both filters

In [50]:
print(f"Unique number of patients who died not considering only albumin tests: {admission_death_no_filter['subject_id'].unique()}")

Unique number of patients who died not considering only albumin tests: [10004720 10006053 10007818 10037975 10038081]


In [51]:
print(f"Unique number of patients who died not considering only albumin tests: {admission_death_no_filter['subject_id'].unique()}")

Unique number of patients who died not considering only albumin tests: [10004720 10006053 10007818 10037975 10038081]
